In [28]:
import pandas as pd
import requests
import json
from datetime import datetime
import time
from sqlalchemy import create_engine
from pandas.io.json import json_normalize

In [29]:
url="https://maps2.dcgis.dc.gov/dcgis/rest/services/FEEDS/MPD/MapServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json"
print(requests.get(url))

<Response [200]>


In [30]:
dc_data = requests.get(url).json()
# print(json.dumps(dc_data, indent=4, sort_keys=True))

In [31]:
ccn = []
district=[]
start_date=[]
offense=[]
latitude=[]
longitude=[]
shift=[]

for features in dc_data["features"]:
    ccn.append(features[u"attributes"][u'CCN'])
    district.append(features[u"attributes"][u"DISTRICT"])
    start_date.append(features[u"attributes"][u"START_DATE"])
    offense.append(features[u"attributes"][u"OFFENSE"])
    latitude.append(features[u"attributes"][u"LATITUDE"])
    longitude.append(features[u"attributes"][u"LONGITUDE"])
    shift.append(features[u"attributes"][u"SHIFT"])
    
df_dc=pd.DataFrame([ccn, district, start_date, offense, latitude, longitude, shift]).T

df_dc.set_axis(["ccn", "district", "start_date",
             "offense","latitude","longitude", "shift"], axis='columns', inplace=True)
df_dc.head()


,ccn,district,start_date,offense,latitude,longitude,shift
0,10199597,1,1573123012000,THEFT/OTHER,38.8876,-76.9829,DAY
1,17084415,7,1495234733000,HOMICIDE,38.8205,-77.0104,MIDNIGHT
2,19012540,7,1548183447000,ROBBERY,38.8332,-76.9975,EVENING
3,19012550,4,1548184852000,THEFT/OTHER,38.9413,-77.0254,EVENING
4,19012562,2,1548180047000,THEFT F/AUTO,38.9034,-77.062,EVENING


In [34]:
df_dc.dtypes

ccn           object
district      object
start_date    object
offense       object
latitude      object
longitude     object
shift         object
dtype: object

In [35]:
connection_string = "postgres:postgres@localhost:5432/dc_crime"
engine = create_engine(f'postgresql://{connection_string}')
engine.table_names()

['df_dc']

,ccn,district,start_date,offense,latitude,longitude,shift
0,10199597,1,1573123012000,THEFT/OTHER,38.887592,-76.982941,DAY
1,17084415,7,1495234733000,HOMICIDE,38.820461,-77.010375,MIDNIGHT
2,19012540,7,1548183447000,ROBBERY,38.833209,-76.997512,EVENING
3,19012550,4,1548184852000,THEFT/OTHER,38.941334,-77.025388,EVENING
4,19012562,2,1548180047000,THEFT F/AUTO,38.903422,-77.061967,EVENING
